In [1]:
import itertools
import numpy as np
import pyquil.api as api
from pyquil.gates import *
from pyquil.quil import Program
from gaussian_elimination import *

##### Problem Setup

The setup for Simon's problem consists of a given black-box operator that is a generalization from those given in the Deutsch and Deutsch-Jozsa problems, and maps $\mathbf{f}: \{0, 1\}^n \rightarrow \{0, 1\}^m$, such that<br>
<br>
$$ U_f : \left\vert \mathbf{x} \right\rangle \left\vert \mathbf{b} \right\rangle \rightarrow \left\vert \mathbf{x} \right\rangle \left\vert \mathbf{b} \oplus \mathbf{f}(\mathbf{x}) \right\rangle$$
<br>
where $\mathbf{f}(\mathbf{x}) \in \{0, 1\}^m \, \, \forall \mathbf{x} \in \{ 0, 1 \}^n$, $\mathbf{b} \in \{0, 1\}^m$, and the $\oplus$ sign represents mod 2 addition on each of the components separately. The problem consists of finding 
$\mathbf{s} \in \{0, 1\}^n$ such that<br>
<br>
$$\mathbf{f} (\mathbf{x} \oplus \mathbf{s}) = \mathbf{f} (\mathbf{x})$$
so that the function $\mathbf{f}$ is periodic with period $\mathbf{s}$.

We solve by first preparing the state $\left\vert\mathbf{x} \right\rangle \left\vert 0 \right\rangle$, applying the black-box to produce the state $\left\vert\mathbf{x}\right\rangle \left\vert \mathbf{f}(\mathbf{x})\right\rangle$, then applying $H^{\otimes n}$ to the first register $\left\vert\mathbf{x}\right\rangle$, then measuring it and recording the value $\mathbf{w}_i$, repeating these steps until $\text{span}\{\mathbf{w}_i\}$ equals $n-1$, at which point we solve the equation $\mathbf{W}\mathbf{s}^{T} = \mathbf{0}^{T}$ via Gaussian elimination to obtain $\mathbf{s}$ as the unique non-zero solution. To see _why_ this works, the reader is referred to "An introduction to quantum computing" by P. Kaye et al.

##### Implementation Notes

We can generalize the black-box operator from the Deutsch-Jozsa problem to construct the one required here
$$U_f = \sum_{\mathbf{x}=0}^{2^{n} - 1} \left\vert \mathbf{x} \right\rangle \left\langle \mathbf{x} \right\vert \otimes \left[ I + f_{i} (\mathbf{x}) \left( X - I \right) \right]^{\otimes_{i=m-1}^{i=0}}$$
For example, if $m=2$, then
$$ \left[ I + f_{i} (\mathbf{x}) \left( X - I \right) \right]^{\otimes_{i=m-1}^{i=0}} = \left[ I + f_1(\mathbf{x}) \left( X - I \right) \right] \otimes \left[ I + f_0(\mathbf{x}) \left( X - I \right)\right]$$
<br>
and further if $n=3$, $\mathbf{x} = 010$, and $\mathbf{f}(\mathbf{x}) = 10$, then
$$ \left[ I + f_{i} (\mathbf{x}) \left( X - I \right) \right]^{\otimes_{i=m-1}^{i=0}} = \left[ I + f_1(010) \left( X - I\right)\right] \otimes \left[ I + f_0(010) \left( X - I\right)\right] \\
= \left[ I + (1)(X-I)\right] \otimes \left[ I + (0) (X-I)\right] \\
= X \otimes I$$
<br>
The sampling of the $\mathbf{w}_{i}$ is done in such a way as to ensure the reduced row-echelon form of the collective $\mathbf{W}$ matrix (note that since we're working with mod 2 arithmetic, we automatically have reduced row-echelon, and not just row-echelon form). Back-substitution is modified to work with mod 2 arithmetic. The entire process is implemented in gaussian_elimination.py, and for an excellent discussion of the mathematical details involved, the reader is referred to Section 18.13 of "<c|Q|c> : A Course in Quantum Computing (for the Community College)", Vol. 1  by Michael Loceff.

### Simon's Algorithm using (n+m) qubits

In [2]:
def qubit_strings(n):
    qubit_strings = []
    for q in itertools.product(['0', '1'], repeat=n):
        qubit_strings.append(''.join(q))
    return qubit_strings

In [3]:
def black_box_map(n, m, s):
    """
    Black-box map f:{0,1}^n -> {0,1}^m, randomly taking values,
    and periodic with period s
    """
    # ensure s lives in {0,1}^n
    if len(s) != n:
        raise AssertionError("Length of period vector should equal n")
    # control qubits
    cont_qubs = qubit_strings(n)
    # target qubits
    targ_qubs = qubit_strings(m)
    
    # initialize empty dictionary to store map values
    d_blackbox = {}
    # initialize counter over control qubits
    i = 0
    # randomly select values from {0,1}^m for the periodic function
    while set(cont_qubs) - set(d_blackbox.keys()) != set():
        # pick a random target
        rand_targ = np.random.choice(targ_qubs)
        # set the same value for x and x + s
        d_blackbox[cont_qubs[i]] = rand_targ
        d_blackbox[add_vec_mod2(cont_qubs[i], s)] = rand_targ
        # avoid iterating over keys already assigned values
        while cont_qubs[i] in d_blackbox.keys():
            i = i + 1
            if i >= n:
                break
        
    return d_blackbox

In [4]:
def qubit_ket(qub_string):
    """
    Form a basis ket out of n-bit string specified by the input 'qub_string', e.g.
    '001' -> |001>
    """
    e0 = np.array([[1], [0]])
    e1 = np.array([[0], [1]])
    d_qubstring = {'0': e0, '1': e1}

    # initialize ket
    ket = d_qubstring[qub_string[0]]
    for i in range(1, len(qub_string)):
        ket = np.kron(ket, d_qubstring[qub_string[i]])
    
    return ket

In [5]:
def projection_op(qub_string):
    """
    Creates a projection operator out of the basis element specified by 'qub_string', e.g.
    '101' -> |101> <101|
    """
    ket = qubit_ket(qub_string)
    bra = np.transpose(ket)  # all entries real, so no complex conjugation necessary
    proj = np.kron(ket, bra)
    return proj

In [6]:
def black_box(n, m, s):
    """
    Inputs:-
    n: no. of control qubits
    m: no. of target qubits
    s: bit-string equal to the period of the black-box map
    
    Output:-
    Unitary representation of the black-box operator
    """
    d_bb = black_box_map(n, m, s)
    # initialize unitary matrix
    N = 2**(n+m)
    unitary_rep = np.zeros(shape=(N, N))
    # populate unitary matrix
    for k, v in d_bb.items():
        # initialize target qubit operator
        targ_op = np.eye(2) + int(v[0])*(-np.eye(2) + np.array([[0, 1], [1, 0]]))
        # fill out the rest of the target qubit operator
        for i in range(1, m):
            cont_op = np.eye(2) + int(v[i])*(-np.eye(2) + np.array([[0, 1], [1, 0]]))
            targ_op = np.kron(targ_op, cont_op)
        # complete the unitary operator for current control qubit-register
        unitary_rep += np.kron(projection_op(k), targ_op)
    
    return unitary_rep

In [7]:
qvm = api.QVMConnection()
# pick number of control qubits to be used
n = 4
# pick number of target qubits to be used
m = 2
# specify the period as an n bit-string
s = '1011'
# make sure s has the correct length
if len(s) != n:
    raise ValueError("s does not have correct bit-string length")
# make sure s is non-zero
if s == '0' * n:
    raise ValueError("s should not be zero vector")
# create the unitary black_box operator
blackbox = black_box(n, m, s)
# initialize the augmented matrix to be solved via Gaussian elimination
W = []
# initialize counter
counter = 0
# run main loop
while rank(W) < n-1:
    # initialize the program
    p = Program()

    # Define U_f
    p.defgate("U_f", blackbox)

    # Prepare the initial state (1/sqrt[2])*(|0> + |1>)^(\otimes n) \otimes |0>^(\otimes m)
    for m_ in range(m):
        p.inst(I(m_))
    for n_ in range(m, n+m):
        p.inst(H(n_))

    # Apply U_f
    p.inst(("U_f",) + tuple(range(n+m)[::-1]))

    # Apply final H^(\otimes n)
    for n_ in range(m, n+m):
        p.inst(H(n_))

    # Final measurement
    classical_regs = list(range(n))
    for i, n_ in enumerate(list(range(m, n+m))[::-1]):
        p.measure(n_, classical_regs[i])

    measure_n_qubits = qvm.run(p, classical_regs)

    # flatten out list
    z = [item for sublist in measure_n_qubits for item in sublist]
    z.append(0)
    
    # add (or not) the new sample z to W
    W = new_sample(W, z)
            
    # increment counter
    counter = counter + 1
    del p
    
print ("The period vector is found to be: ", solve_reduced_row_echelon_form(W))

The period vector is found to be:  [1, 0, 1, 1]
